#   Chapter 5: Propensity Score

## 5.1. Import Libraries

In [1]:
import numpy as np 
import pandas as pd

from joblib import Parallel, delayed
from patsy import dmatrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.formula import api as smf

In [2]:
df = pd.read_csv('data/management_training.csv')
df.head()

,departament_id,intervention,engagement_score,tenure,n_of_reports,gender,role,last_engagement_score,department_score,department_size
0,76,1,0.277359,6,4,2,4,0.614261,0.224077,843
1,76,1,-0.449646,4,8,2,4,0.069636,0.224077,843
2,76,1,0.769703,6,4,2,4,0.866918,0.224077,843
3,76,1,-0.121763,6,4,2,4,0.029071,0.224077,843
4,76,1,1.526147,6,4,1,4,0.589857,0.224077,843


## 5.2. Adjusting with Regression


In [3]:
# basic model
print(smf.ols('engagement_score ~ intervention', data=df).fit().summary().tables[1])

                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.2347      0.014    -16.619      0.000      -0.262      -0.207
intervention     0.4346      0.019     22.616      0.000       0.397       0.472


In [4]:
# adding covariates 
model = smf.ols("""
engagement_score ~ intervention 
+ tenure 
+ last_engagement_score 
+ department_score 
+ n_of_reports 
+ C(gender) 
+ C(role)
""", data=df).fit()
print(model.summary()) 
print(' ')
print(' ')
print('Average Treatment Effect (ATE):', model.params['intervention'])
print('95% CI:', model.conf_int().loc['intervention', :].values.T)

                            OLS Regression Results                            
Dep. Variable:       engagement_score   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     357.8
Date:                Tue, 14 May 2024   Prob (F-statistic):               0.00
Time:                        01:47:44   Log-Likelihood:                -13205.
No. Observations:               10391   AIC:                         2.643e+04
Df Residuals:                   10380   BIC:                         2.651e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -1.72

In [5]:
print('Average Treatment Effect (ATE):', model.params['intervention'])
print('95% CI:',  model.conf_int().loc['intervention', :].values.T)

Average Treatment Effect (ATE): 0.2677908576676856
95% CI: [0.23357751 0.30200421]


## 5.3. Propensity Score
- You don't need to control for confounders to achieve conditional independence. Instead, you can control for the conditional probability of treatment (a balancing score, aka, a propensity score)

- A propensity score also serves as a dimensionality reduction technique that helps to block backdoor paths

- The propensity score is the propensity to receive the treatment

- If you take two samples, one from treated and one from the control group, but with same probability of receiving the treatment, they are comparable. Since they have same probability of receiving the treatment, the only reason one of them received the treatment and the other did not is by pure chance (i.e., as good as random) 

### 5.3.1. Propensity Score Estimation

In [6]:
propensity_score_model = smf.logit("""
intervention ~ tenure 
+ last_engagement_score
+ department_score
+ C(n_of_reports)
+ C(gender)
+ C(role)
""", data=df).fit(disp=0)
print(propensity_score_model.summary()) 

                           Logit Regression Results                           
Dep. Variable:           intervention   No. Observations:                10391
Model:                          Logit   Df Residuals:                    10375
Method:                           MLE   Df Model:                           15
Date:                Tue, 14 May 2024   Pseudo R-squ.:                 0.04063
Time:                        01:47:44   Log-Likelihood:                -6877.9
converged:                       True   LL-Null:                       -7169.2
Covariance Type:            nonrobust   LLR p-value:                1.733e-114
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -1.8400      0.141    -13.016      0.000      -2.117      -1.563
C(n_of_reports)[T.2]      0.0515      0.086      0.600      0.548      -0.117       0.220
C(n_of_r

In [7]:
propensity_score_data = df.assign(
    propensity_score = propensity_score_model.predict(df)
)
propensity_score_data.head()

,departament_id,intervention,engagement_score,tenure,n_of_reports,gender,role,last_engagement_score,department_score,department_size,propensity_score
0,76,1,0.277359,6,4,2,4,0.614261,0.224077,843,0.596106
1,76,1,-0.449646,4,8,2,4,0.069636,0.224077,843,0.391138
2,76,1,0.769703,6,4,2,4,0.866918,0.224077,843,0.602578
3,76,1,-0.121763,6,4,2,4,0.029071,0.224077,843,0.580990
4,76,1,1.526147,6,4,1,4,0.589857,0.224077,843,0.619976


In [8]:
# check performance by intervention
propensity_score_data.groupby(['intervention'])['propensity_score'].agg(['mean', 'median'])

,mean,median
intervention,,
0,0.510464,0.517268
1,0.565136,0.574800


Notes on using ML models for Propensity Score Matching
- Ensure that the predictions are calibrated probabilities
- Ensure that out-of-fold predictions are used to avoid bias due to overfitting

### 5.3.2. Propensity Score and Orthogonalization
You can use propensity score directly in a linear regression to predict treatment effect

In [9]:
model = smf.ols("""
engagement_score ~ intervention 
+ propensity_score
""", data=propensity_score_data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:       engagement_score   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                     1084.
Date:                Tue, 14 May 2024   Prob (F-statistic):               0.00
Time:                        01:47:44   Log-Likelihood:                -13759.
No. Observations:               10391   AIC:                         2.752e+04
Df Residuals:                   10388   BIC:                         2.755e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.8339      0.042  

### 5.3.3. Propensity Score Matching
You can use propensity score directly in a linear regression to predict treatment effect

In [10]:
T = 'intervention'
X = 'propensity_score'
Y = 'engagement_score'

treated = propensity_score_data.query(f'{T}==1')
untreated = propensity_score_data.query(f'{T}==0')

mt0 = KNeighborsRegressor(n_neighbors=1).fit(untreated[[X]], untreated[Y])
mt1 = KNeighborsRegressor(n_neighbors=1).fit(treated[[X]], treated[Y])

predicted = pd.concat(
    [
        # find matches for the treated using the untreated KNN model
        treated.assign(match=mt0.predict(treated[[X]])),
        untreated.assign(match=mt1.predict(untreated[[X]]))
    ]
)

predicted.head()

,departament_id,intervention,engagement_score,tenure,n_of_reports,gender,role,last_engagement_score,department_score,department_size,propensity_score,match
0,76,1,0.277359,6,4,2,4,0.614261,0.224077,843,0.596106,0.557680
1,76,1,-0.449646,4,8,2,4,0.069636,0.224077,843,0.391138,-0.952622
2,76,1,0.769703,6,4,2,4,0.866918,0.224077,843,0.602578,-0.618381
3,76,1,-0.121763,6,4,2,4,0.029071,0.224077,843,0.580990,-1.404962
4,76,1,1.526147,6,4,1,4,0.589857,0.224077,843,0.619976,0.000354


In [11]:
np.mean(
    (predicted[Y] - predicted['match']) * predicted [T]
    + (predicted['match'] - predicted[Y]) * (1-predicted[T])
)

0.28777443474045966

### 5.3.4. Inverse Propensity Weighting

In [12]:
weight_t = 1/propensity_score_data.query('intervention==1')['propensity_score']
weight_nt = 1/(1-propensity_score_data.query('intervention==0')['propensity_score'])
t1 = propensity_score_data.query('intervention==1')['engagement_score']
t0 = propensity_score_data.query('intervention==0')['engagement_score']

y1 = sum(t1*weight_t)/len(propensity_score_data)
y0 = sum(t0*weight_nt)/len(propensity_score_data)

print('E[Y1]:', y1)
print('E[Y0]:', y0)
print('ATE', y1  - y0)

E[Y1]: 0.11656317232946742
E[Y0]: -0.14941553647814415
ATE 0.26597870880761154


### 5.3.5. Variance of IPW

In [13]:
def estimate_ate_with_propensity_score(df, propensity_score_formula, T, Y): 
    
    X = dmatrix(propensity_score_formula, df)
    ps_model = LogisticRegression(penalty=None, max_iter=1000).fit(X, df[T])
    ps = ps_model.predict_proba(X)[:, 1]
    
    return np.mean((df[T]-ps)/ (ps*(1-ps)) * df[Y])

In [14]:
formula = """tenure + last_engagement_score + department_score
+ C(n_of_reports) + C(gender) + C(role)
"""
T = 'intervention'
Y = 'engagement_score'

print(estimate_ate_with_propensity_score(df, formula, T, Y))

0.26601327333120944


In [15]:
print(f'ATE: {estimate_ate_with_propensity_score(df, formula, T, Y) }')

ATE: 0.26601327333120944
